In [1]:
import pandas as pd

In [2]:
wehealth_tranining = pd.read_excel('wehealth.xlsx')

In [3]:
wehealth_tranining.head(10)

,Sympt_1,Sympt_2,Sympt_3,Sympt_4,Sympt_5,Disease
0,fever,headache,vomiting,muscle pain,shaking chills,Malaria
1,fever,headache,vomiting,muscle pain,Skin rash,Loda
2,fever,runny nose,sore throat,muscle pain,cough,Viral
3,fever,runny nose,sore throat,fatigue,sneezing,Viral
4,fever,headache,vomiting,muscle pain,eye pain,Dengu
5,abdominal pain,headache,muscle pain,fatigue,shaking chills,Malaria


In [4]:
#check unique values
def uniqueValues(df,col):
    return df[col].unique()

In [5]:
#uniqueValues(wehealth_tranining,'Sympt_1')
uniqueValues(wehealth_tranining,'Sympt_2')

array(['headache', 'runny nose'], dtype=object)

In [6]:
X = wehealth_tranining.drop('Disease',axis=1)

In [7]:
y= wehealth_tranining['Disease']
y

0    Malaria
1       Loda
2      Viral
3      Viral
4      Dengu
5    Malaria
Name: Disease, dtype: object

In [8]:
def class_counts(df):
    counts={}
    for rows in df.iloc[:,-1]:
        if rows not in counts:
            counts[rows] = 0
        counts[rows]+=1
    return counts

In [9]:
class_counts(wehealth_tranining)

{'Malaria': 2, 'Loda': 1, 'Viral': 2, 'Dengu': 1}

In [10]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [11]:
is_numeric(1)

True

In [12]:
class Questions:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def matchs(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        #print("columns .    ",self.column, "example  " ,example)
        val = example[self.column]
        #print(val ,"  =======  ",self.value, "columns .    ",self.column, "example  " ,example)
        #if is_numeric(val):
         #   return val >= self.value
        #else:
        return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
  #      if is_numeric(self.value):
   #         condition = ">="
        return "Do you have %s ?" % (str(self.value))

In [13]:
q = Questions(X.Sympt_1,'fever')
q

Do you have fever ?

In [14]:
def partition(df, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    #print ("df .  ",df)
    true_branch, false_branch = [], []
    for index, row in df.iterrows():
        if question.matchs(row):
            true_branch.append(row)
        else:
            false_branch.append(row)
    return true_branch, false_branch

In [15]:
true_branch, false_branch = partition(wehealth_tranining,Questions(0,'fever'))

In [16]:
true_df = pd.DataFrame(true_branch,columns=None)
true_df

,Sympt_1,Sympt_2,Sympt_3,Sympt_4,Sympt_5,Disease
0,fever,headache,vomiting,muscle pain,shaking chills,Malaria
1,fever,headache,vomiting,muscle pain,Skin rash,Loda
2,fever,runny nose,sore throat,muscle pain,cough,Viral
3,fever,runny nose,sore throat,fatigue,sneezing,Viral
4,fever,headache,vomiting,muscle pain,eye pain,Dengu


In [17]:
false_df = pd.DataFrame(false_branch).head()
false_df

,Sympt_1,Sympt_2,Sympt_3,Sympt_4,Sympt_5,Disease
5,abdominal pain,headache,muscle pain,fatigue,shaking chills,Malaria


In [18]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    #print("count   ",counts)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [19]:
pure = pd.DataFrame()
pure = wehealth_tranining[{'Sympt_1'}]

In [20]:
# this will return 0
gini(pure)

0.2777777777777777

In [21]:
def information_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [22]:
current_uncertainty = gini(wehealth_tranining)
current_uncertainty

0.7222222222222222

In [23]:
# How much information do we gain by partioning on 'fever'?
true_rows, false_rows = partition(wehealth_tranining, Questions(1, 'headache'))
information_gain(pd.DataFrame(true_rows), pd.DataFrame(false_rows), current_uncertainty)

0.3055555555555556

In [24]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    #print("current_uncertainty    ",current_uncertainty)
    
    n_features = len(rows.columns)-1  # number of columns
    
    #print("rows    ",rows)
    #print("n_features    ",n_features) 

    for col in range(n_features):  # for each feature
        #print("col " , rows.iloc[:,col])
        #print("col .  ",col)
        values = rows.iloc[:,col].unique()  # unique values in the column
        #print("values .  ",values)

        for val in values:  # for each value

            question = Questions(col, val)
            #print("question  ",question)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = information_gain(pd.DataFrame(true_rows), pd.DataFrame(false_rows), current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [25]:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(wehealth_tranining)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Do you have headache ?

In [26]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [27]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [28]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)
   # print ("gain ",gain)
    #print ("question ",question)
    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(pd.DataFrame(true_rows))

    # Recursively build the false branch.
    false_branch = build_tree(pd.DataFrame(false_rows))
    
    #print("true rows .  ",true_rows)
    #print("false rows .  ",false_rows)
    
    #print ("true_branch ",true_branch)
    #print ("false_branch ",false_branch)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [29]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [30]:
my_tree = build_tree(wehealth_tranining)

In [31]:
print_tree(my_tree)

Do you have headache ?
--> True:
  Do you have shaking chills ?
  --> True:
    Predict {'Malaria': 2}
  --> False:
    Do you have eye pain ?
    --> True:
      Predict {'Dengu': 1}
    --> False:
      Predict {'Loda': 1}
--> False:
  Predict {'Viral': 2}


In [32]:
def classify(row, node):
    """See the 'rules of recursion' above."""
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    print(row[1])
    if node.question.matchs(row):
        #print("row . ",row )
        
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [33]:
classify(wehealth_tranining.iloc[:,0], my_tree)

fever


{'Viral': 2}

In [34]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [35]:
print_leaf(classify(wehealth_tranining.iloc[:,0], my_tree))

fever


{'Viral': '100%'}

In [36]:
print_leaf(classify(wehealth_tranining.iloc[:,1], my_tree))

headache
headache
headache


{'Loda': '100%'}

In [37]:
wehealth_testing = pd.read_excel('wehealth_testing.xlsx')

In [38]:
for index,row in wehealth_testing.iterrows():
    #print("row   ",row)
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

headache
headache
headache
Actual: Skin rash. Predicted: {'Loda': '100%'}
runny nose
Actual: cough. Predicted: {'Viral': '100%'}
headache
headache
Actual: shaking chills. Predicted: {'Malaria': '100%'}


In [39]:
wehealth_testing

,Sympt_1,Sympt_2,Sympt_3,Sympt_4,Sympt_5
0,fever,headache,vomiting,muscle pain,Skin rash
1,fever,runny nose,sore throat,muscle pain,cough
2,abdominal pain,headache,muscle pain,anemia,shaking chills


In [40]:
user_input = [None]*4
def user_interaction(rows):
    gain, question = find_best_split(rows)
   # print(gain, question)
    if gain == 0:
        print(Leaf(rows).predictions)
        return
    val = input(question)
    #user_input.insert(question.column, val)
    true_rows, false_rows = partition(rows, question)
    if val.lower() == 'yes':
        user_input.insert(question.column, question.value)
        user_interaction(pd.DataFrame(true_rows))
    elif val.lower() == 'no':
        user_input.insert(question.column, question.value)        
        user_interaction(pd.DataFrame(false_rows))
    #print(user_input)
    return user_input
#if __name__ == '__main__':
#     answer = user_interaction(wehealth_tranining)
#     answer_df = pd.DataFrame(answer)
#     #print(answer_df)
#     print(classify(answer_df, my_tree))

In [41]:
user_interaction(wehealth_tranining)

0.3055555555555556 Do you have headache ?
Do you have headache ?yes
0.375 Do you have shaking chills ?
Do you have shaking chills ?no
0.5 Do you have eye pain ?
Do you have eye pain ?yes
0 None
{'Dengu': 1}


[None, 'headache', None, None, 'eye pain', 'shaking chills', None]